In [442]:
with open('22a.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]
lines = [line for line in lines if len(line) > 2]

In [443]:
import re
from typing import Tuple

In [444]:
class Command:

    def __init__(self, line=None, coords=None):
        if line != None:
            self.line = line
            self.on = 1 if line[1] == 'n' else 0
            splitted = re.split('=|,|\.', line)
            self.off_x_min, self.off_x_max, self.off_y_min, self.off_y_max, self.off_z_min, self.off_z_max = (-100000,-100000,-100000,-100000,-100000,-100000)
            self.x_min = int(splitted[1])
            self.x_max = int(splitted[3])
            self.y_min = int(splitted[5])
            self.y_max = int(splitted[7])
            self.z_min = int(splitted[9])
            self.z_max = int(splitted[11])
        elif coords != None:
            self.x_min, self.x_max, self.y_min, self.y_max, self.z_min, self.z_max, self.on = coords
    
        self.volume = abs(self.x_max + 1 - self.x_min) * abs(self.y_max + 1 - self.y_min) * abs(self.z_max + 1 - self.z_min)
        self.inner = True
        if self.x_min < -50 or self.x_max > 50 or self.y_min < -50 or self.y_max > 50 or self.z_min < -50 or self.z_max > 50:
            self.inner = False
         
    def __str__(self):
        return self.line

    def overlap(self, other_list):
        
        counter = 0
        new_list = []
        for other in other_list:
            x_ovl = self.x_max >= other.x_min and other.x_max >= self.x_min
            y_ovl = self.y_max >= other.y_min and other.y_max >= self.y_min
            z_ovl = self.z_max >= other.z_min and other.z_max >= self.z_min
            ovl = x_ovl and y_ovl and z_ovl
            if ovl:
                if other.on:
                    off_x_min = max(self.x_min, other.x_min)
                    off_x_max = min(self.x_max, other.x_max)
                    off_y_min = max(self.y_min, other.y_min)
                    off_y_max = min(self.y_max, other.y_max)
                    off_z_min = max(self.z_min, other.z_min)
                    off_z_max = min(self.z_max, other.z_max)
                    off_zone = Command(None, (off_x_min,off_x_max,off_y_min,off_y_max,off_z_min,off_z_max,0))
                    new_list.append(off_zone)
                    counter -= off_zone.volume
                else:
                    if self.on: 
                        off_x_min = max(self.x_min, other.x_min)
                        off_x_max = min(self.x_max, other.x_max)
                        off_y_min = max(self.y_min, other.y_min)
                        off_y_max = min(self.y_max, other.y_max)
                        off_z_min = max(self.z_min, other.z_min)
                        off_z_max = min(self.z_max, other.z_max)
                        off_zone = Command(None, (off_x_min,off_x_max,off_y_min,off_y_max,off_z_min,off_z_max,1))
                        new_list.append(off_zone)
                        counter += off_zone.volume
                    else:
                        on_x_min = max(self.x_min, other.x_min)
                        on_x_max = min(self.x_max, other.x_max)
                        on_y_min = max(self.y_min, other.y_min)
                        on_y_max = min(self.y_max, other.y_max)
                        on_z_min = max(self.z_min, other.z_min)
                        on_z_max = min(self.z_max, other.z_max)
                        on_zone = Command(None, (on_x_min,on_x_max,on_y_min,on_y_max,on_z_min,on_z_max,1))
                        new_list.append(on_zone)
                        counter += on_zone.volume
        if self.on: 
            new_list.append(self)
            counter += self.volume
        return counter, other_list + new_list
        


In [445]:
commands = [Command(line) for line in lines]
inner_commands = [cmd for cmd in commands if cmd.inner]

In [446]:
def coordinate_shifter(row: int, col: int, dep: int) -> Tuple[int, int, int]:
    return (row + 50, col + 50, dep + 50)

In [447]:
init_size = 50
size = init_size * 2 + 1

In [448]:
cube = [[[0 for k in range(size)] for j in range(size)] for i in range(size)]

In [449]:
for cmd in inner_commands:
    for x in range(cmd.x_min, cmd.x_max + 1):
        for y in range(cmd.y_min, cmd.y_max + 1):
            for z in range(cmd.z_min, cmd.z_max + 1):
                cube[x][y][z] = cmd.on

In [450]:
def count_on(cube) -> int:
    counter = 0
    size = len(cube)
    for i in range(size):
        for j in range(size):
            for k in range(size):
                counter += cube[i][j][k]
    return counter

In [451]:
print(count_on(cube))

650099


### Part 2

In [452]:
on_counter = 0
ls = []
for cmd in commands:
    curr, ls = cmd.overlap(ls)
    on_counter += curr
print(on_counter)

1254011191104293
